In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
from funcs import read_config, get_groups, pull_clean, get_kpi2_respuestas, get_kpi3_retencion, get_kpi4_influencia

2023-12-09 16:15:11.470 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
#Data Pull and Functions
data_frames = pull_clean() 
states= data_frames.get('1_states.parquet')
categories = data_frames.get('2_categories.parquet')
# user_yelp = data_frames.get('3_user_yelp.parquet')
user_google = data_frames.get('4_user_google.parquet')
business_google = data_frames.get('5_business_google.parquet')
# business_yelp = data_frames.get('6_business_yelp.parquet')
categories_google = data_frames.get('7_categories_google.parquet')
# categories_yelp = data_frames.get('8_categories_yelp.parquet')
reviews_google = data_frames.get('9_reviews_google.parquet')
# reviews_yelp = data_frames.get('10_reviews_yelp.parquet')
groups_google = data_frames.get('11_grupo_de_categorias_google.parquet')
groups_yelp = data_frames.get('12_grupo_de_categorias_yelp.parquet')
df_user = data_frames.get('user_categories')
df_categories = data_frames.get('locales_categories')

2023-12-09 16:15:11.749 
  command:

    streamlit run /home/p/Code/Henry/PI_DA/.env/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-12-09 16:15:11.761 No runtime found, using MemoryCacheStorageManager


1_states.parquet: (4, 2)
2_categories.parquet: (1297, 2)
4_user_google.parquet: (1360619, 5)
5_business_google.parquet: (24755, 6)
6_business_yelp.parquet: (12917, 6)
7_categories_google.parquet: (70307, 2)
9_reviews_google.parquet: (2430142, 8)
11_grupo_de_categorias_google.parquet: (29183, 2)
user_categories: (1713864, 3)
locales_categories: (132176, 3)


In [8]:
states

,state_id,state
0,1,Florida
1,2,California
2,3,Illinois
3,4,New Jersey


In [9]:
reviews_google

,review_id,user_id,gmap_id,stars,sentiment,date,resp_sentiment,resp_date
0,0,1.0899115226265579e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,NaT,0.0,NaT
1,1,1.1129032221979622e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,NaT,0.0,NaT
2,2,1.1264035744961195e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,4,0,NaT,0.0,NaT
3,3,1.1744034972382365e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,NaT,0.0,NaT
4,4,1.0058077083612353e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,NaT,0.0,NaT
...,...,...,...,...,...,...,...,...
9599941,2452767,1.0910540659763497e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,4,0,NaT,0.0,NaT
9599942,2452768,1.1598048245262603e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,4,0,NaT,0.0,NaT
9599943,2452769,1.159471047965541e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,4,0,NaT,0.0,NaT
9599944,2452770,1.1792358760866297e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,3,0,NaT,0.0,NaT


In [4]:
unique_groups = groups_google['group'].unique()
print(unique_groups) 

['general' 'fast food' 'bars & nightlife' 'foreign' 'coffess & breakfast'
 'veggie & vegetarian']
